# Evaluation: Completeness

Part II of the computational evaluation of AI-generated linked data for [Linking Anthropology's Data and Archives (LADA)](https://ischool.umd.edu/projects/building-a-sustainable-future-for-anthropologys-archives-researching-primary-source-data-lifecycles-infrastructures-and-reuse/), focused on completeness (e.g., metadata fields are not empty or 'unknown,' URLs in field values are valid).

---

**Table of Contents:**

I. [Data Loading](#data-loading)

II. [Completeness](#completeness)

  * [Content of Fields](#content-of-fields)

    * [Dublin Core](#dublin-core)

    * [JSON-LD](#json-ld)

---

## Data Loading

In [ ]:
import utils
import config
import pandas as pd
import urllib
from urllib.request import Request, urlopen
import xml.etree.ElementTree as ET
import json
from pyld import jsonld
from lxml import etree
from pathlib import Path
import os
import re

Create variables to reference existing directories and files.

In [ ]:
dublin_path = "cleaned/dublin_core/"  # XML data files
schema_path = "cleaned/schema_org/"   # JSON data files
cidoc_path = "cleaned/cidoc_crm/"     # JSON data files

dublin_t1_dir = config.task1_data+dublin_path
schema_t1_dir = config.task1_data+schema_path
cidoc_t1_dir = config.task1_data+cidoc_path

dublin_p1_dir = config.playgrd1_data+dublin_path
schema_p1_dir = config.playgrd1_data+schema_path
cidoc_p1_dir = config.playgrd1_data+cidoc_path

dublin_p3_dir = config.playgrd3_data+dublin_path
schema_p3_dir = config.playgrd3_data+schema_path
cidoc_p3_dir = config.playgrd3_data+cidoc_path

Create a directory to store the error reports in.

In [ ]:
d = "completeness"
report_dir = f"data/error_reports/{d}/"
Path(report_dir).mkdir(parents=True, exist_ok=True)

For checking URL vaildity:

In [ ]:
os.environ["no_proxy"] = "*"                                                                                                                     # https://docs.python.org/3/library/urllib.request.html 
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}   # As suggested here: https://www.reddit.com/r/learnpython/comments/1ea3r0z/how_to_avoid_http_error_403_forbidden/

## Content of Fields

Review the Dublin Core XML, Schema.org JSON-LD, and CIDOC-CRM JSON-LD metadata records to identify fields without values or where uncertainty about the field's value is expressed (e.g., `unknown`, `not specified`).

### Dublin Core

In [ ]:
# Read the TXT files so all generated metadata can be read, whether or not the XML is well-formed.
extension = ".txt"
dublin_file_paths = []
dublin_files_t1 = [f for f in os.listdir(dublin_t1_dir) if f.endswith(extension)]
dublin_file_paths += [dublin_t1_dir+f for f in dublin_files_t1]
dublin_files_p1 = [f for f in os.listdir(dublin_p1_dir) if f.endswith(extension)]
dublin_file_paths += [dublin_p1_dir+f for f in dublin_files_p1]
dublin_files_p3 = [f for f in os.listdir(dublin_p3_dir) if f.endswith(extension)]
dublin_file_paths += [dublin_p3_dir+f for f in dublin_files_p3]
dublin_file_paths.sort()
total_dc_files = len(dublin_file_paths)
print(f"Total Dublin Core {extension[1:].upper()} files:", total_dc_files)

#### Empty Fields

Check for empty metadata fields.

In [ ]:
empty = re.compile('(<[a-z]+:[a-z]+>|<[a-z=" ]+>)((unknown|none|na|""|\?|not specified|\n|)|[^<>]+(not specified|unknown))(</[a-z]+:[a-z]+>|</[a-z]+>)')

In [ ]:
files_with_empty, empty_fields_per_file, fields_per_file = utils.findEmptyFields(empty, dublin_file_paths)

Create a DataFrame with the empty fields data so we can review it as a table.  We'll sort the data so the files with the most empty fields appear at the top and the files without any empty fields appear at the bottom of the table.

In [ ]:
df_empty = pd.DataFrame.from_dict({"file_path":dublin_file_paths, "empty_field_count":empty_fields_per_file, "fields":fields_per_file}).sort_values(by="empty_field_count", ascending=False)
df_empty.head()

The first five rows of the DataFrame are displayed above and the last five rows of the DataFrame are displayed below.

In [ ]:
df_empty.tail()

In [ ]:
assert df_empty.shape[0] == len(dublin_file_paths), "The new DataFrame should have exactly one row per Dublin Core metadata record (per file)."

Create a report showing how many files have different amounts of empty fields.

In [ ]:
empty_field_count_report = pd.DataFrame(df_empty.empty_field_count.value_counts()).rename(columns={"count":"file_count"})
empty_field_count_report

We can see that 44 files don't have any empty fields, and 63 files have 1 or more empty fields.

"Explode" the DataFrame so that instead of having one row per file, it has one row per metadata field.  For this, we'll remove ("drop") all the files that don't have any empty fields.

In [ ]:
df_empty_exploded = df_empty.loc[df_empty.empty_field_count > 0].drop(columns=["empty_field_count"]).explode("fields")
assert sum(empty_fields_per_file) == df_empty_exploded.shape[0], "There should be exactly one row per empty field."

In [ ]:
df_empty_exploded.head() # Show the first five rows of the exploded DataFrame

Let's transform the data again so the DataFrame has a column for the field with an empty value (i.e., the name or attribute of the XML tag) and a column for the value itself.

In [ ]:
fields = (list(df_empty_exploded.fields))
# Extract the tag name or attribute that indicates the Dublin Core field intended. If the tag
# is 'dc' and the metadata field is provided as an attribute, such as '<dc element="title">',
# then the extracted data will be 'title,' not 'dc.'
tags = [re.search('(?<=<)([a-z:]+)(?=>)|(?<=")[a-z]+(?=")', field)[0] for field in fields]
values_lists = [re.findall('>[^<]*<', field) for field in fields]
values = []
for v in values_lists:
    if len(v) > 0:
        values += [v[0][1:-1]]
    else:
        values += ['']
df_empty_exploded.insert(len(df_empty_exploded.columns), "tag_or_attribute", tags)
df_empty_exploded.insert(len(df_empty_exploded.columns), "empty_value", values)
df_empty_exploded.tail()

Calculate the occurrence of each tag/attribute name.

In [ ]:
tag_counts = pd.DataFrame(df_empty_exploded.tag_or_attribute.value_counts()).reset_index()
tag_counts

In [ ]:
tag_values = list(tag_counts.tag_or_attribute)
tag_cats = []
for t in tag_values:
    if ":" in t:
        tag_cats += [t.split(":")[-1]]
    else:
        tag_cats += [t]
tag_counts.insert(1, "category", tag_cats)
tag_counts

In [ ]:
df_cats = tag_counts.groupby(["category"]).transform("sum")
df_cats.insert(0, "category", tag_counts.category)
df_cats = df_cats.drop(columns=["tag_or_attribute"]).drop_duplicates()
df_cats

Calculate the occurrence of each "empty" value.

In [ ]:
df_values = pd.DataFrame(df_empty_exploded.empty_value.value_counts())
df_values

Since the new line (`\n`) and empty string (`""`) characters will show up in the exported CSV file as blank table cells, we'll replace the cells in the `empty values` column for clarity: 

In [ ]:
df_values = utils.emptyErrorReportValues(df_values, "empty_value")
df_values

Save the reports as CSV files.

In [ ]:
metadata_standard = "dublin_core"
data_serialization = "xml"
report_type = "empty_field_counts"
df_empty.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=True
    )

In [ ]:
metadata_standard = "dublin_core"
data_serialization = "xml"
report_type = "files_per_empty_field_count"
empty_field_count_report.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=True
    )

In [ ]:
metadata_standard = "dublin_core"
data_serialization = "xml"
report_type = "empty_fields_by_file"
df_empty_exploded.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=True
    )

In [ ]:
metadata_standard = "dublin_core"
data_serialization = "xml"
report_type = "empty_field_tag_counts"
tag_counts.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=True
    )

In [ ]:
metadata_standard = "dublin_core"
data_serialization = "xml"
report_type = "empty_field_tag_category_counts"
df_cats.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=True
    )

In [ ]:
metadata_standard = "dublin_core"
data_serialization = "xml"
report_type = "empty_field_value_counts"
df_values.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=True
    )

#### URLs

##### Namespace URLs
First, check that the namespace URLs are well-formed and that they exist.

In [ ]:
url_pattern = re.compile('([a-z]+ns:[a-z]+|[a-z]+ns)=[^>]+( [^>])*(?=>)')

In [ ]:
# Find all the URLs
files_with_urls, url_count_per_file, urls_per_file = [], [], []
for file_path in dublin_file_paths:
    with open(file_path, "r") as f:
        f_string = f.read().lower()
        
        # Look for URLs in the file
        has_urls = re.finditer(url_pattern, f_string)
        # Save the URLs in a list per file
        file_urls = []
        for match in has_urls:
            url = match[0]
            if " " in url:
                multiple = url.split(" ")
                file_urls = file_urls + multiple
                # print(file_urls)
            else:
                file_urls += [url]
        urls_per_file += [file_urls]
        url_count_per_file += [len(file_urls)]
        
        if len(file_urls) > 0:
            # Save the file path to the XML version of the file
            file_path.replace(".txt", ".xml")
            files_with_urls += [file_path]

        f.close()

print(sum(url_count_per_file), "URLs found in", len(files_with_urls), "files.")

In [ ]:
url_df = pd.DataFrame.from_dict({"file_path":dublin_file_paths, "url_count":url_count_per_file, "urls":urls_per_file}).sort_values(by="url_count", ascending=False)
url_df.head()

In [ ]:
url_df = url_df.loc[url_df["url_count"] > 0]  # Keep only files with URLs
url_df_exploded = url_df.explode("urls").drop(columns=["url_count"])
url_df_exploded.head()

In [ ]:
urls = list(url_df_exploded.urls)
print(urls[:10])

Check that each URL is preceded by a namespace and surrounded in quotes (i.e., `xmlns:dc="[URL_GOES_HERE]"`), otherwise the URL was incorrectly included in the metadata record.

In [ ]:
correct_namespace = '([a-z]+ns:[a-z]+|[a-z]+ns)="https?://[a-z0-9\-._~:/?#@!$&\'()*+,;=%]+"'
correct_url = 'https?://[a-z0-9\-._~:/?#@!$&\'()*+,;=%]+'

In [ ]:
valid_namespace, valid_url = [], []
for url in urls:
    if re.match(correct_namespace, url):
        valid_namespace += [True]
    else:
        valid_namespace += [False]
    
    if re.search(correct_url, url):
        valid_url += [True]
    else:
        valid_url += [False]

url_df_exploded.insert(len(url_df_exploded.columns), "valid_namespace_format", valid_namespace)
url_df_exploded.insert(len(url_df_exploded.columns), "valid_url_format", valid_url)
url_df_exploded.tail()

In [ ]:
total_urls = url_df_exploded.shape[0]
print("Total URLs:", total_urls)

In [ ]:
url_status = pd.DataFrame(url_df_exploded.valid_namespace_format.value_counts()).rename(columns={"count":"total_urls"})
proportions = (url_status[["total_urls"]]/total_urls).values
percentages = [f"{proportion[0]*100:.2f}%" for proportion in proportions]
url_status.insert(len(url_status.columns), "proportion_of_urls", percentages)
url_status

In [ ]:
url_status2 = pd.DataFrame(url_df_exploded.valid_url_format.value_counts()).rename(columns={"count":"total_urls"})
proportions = (url_status2[["total_urls"]]/total_urls).values
percentages = [f"{proportion[0]*100:.2f}%" for proportion in proportions]
url_status2.insert(len(url_status2.columns), "proportion_of_urls", percentages)
url_status2

In [ ]:
file_url_status = url_df_exploded.drop(columns=["urls"]).drop_duplicates()
file_url_status = pd.DataFrame(file_url_status.valid_namespace_format.value_counts()).rename(columns={"count":"file_count"})
df_url_status = url_status.join(file_url_status)
proportions = (df_url_status[["file_count"]]/total_dc_files).values
percentages = [f"{proportion[0]*100:.2f}%" for proportion in proportions]
df_url_status.insert(len(df_url_status.columns), "proportion_of_files", percentages)
df_url_status

In [ ]:
file_url_status2 = url_df_exploded.drop(columns=["urls"]).drop_duplicates()
file_url_status2 = pd.DataFrame(file_url_status2.valid_url_format.value_counts()).rename(columns={"count":"file_count"})
df_url_status2 = url_status2.join(file_url_status2)
proportions = (df_url_status2[["file_count"]]/total_dc_files).values
percentages = [f"{proportion[0]*100:.2f}%" for proportion in proportions]
df_url_status2.insert(len(df_url_status2.columns), "proportion_of_files", percentages)
df_url_status2

Extract the URLs provided, even if not in a valid format within a metadata record, and then check whether the URL exists.

In [ ]:
request_errors = []
for url in urls:
    clean = re.findall('https?:\/\/[^>"]+', url)
    if len(clean) > 0:
        clean_url = clean[0]
        clean_url = clean_url.strip('"')
        clean_url = clean_url.strip(' ')
        try:
            url_request = urllib.request.Request(clean_url, headers=headers)
            html = urllib.request.urlopen(url_request, timeout=5).read()
            request_errors += ["No error"]  # Indicates a valid URL (though a manual check is needed to make sure it's a relevant URL)
        except Exception as e:
            request_errors += [str(e)]
    else:
        request_errors += ["Invalid format (no request made)"]
print("Finished requests!")

In [ ]:
url_df_exploded.insert(len(url_df_exploded.columns), "request_error", request_errors)
url_df_exploded.tail()

In [ ]:
url_df_exploded.request_error.value_counts()

In [ ]:
url_errors_df1 = url_df_exploded.loc[url_df_exploded.valid_namespace_format == False]
url_errors_df2 = url_df_exploded.loc[url_df_exploded.valid_url_format == False]
url_errors_df3 = url_df_exploded.loc[url_df_exploded.request_error == "HTTP Error 300: Multiple Choices"]
url_errors_df4 = url_df_exploded.loc[url_df_exploded.request_error == "HTTP Error 404: Not Found"]
url_errors_df = pd.concat([url_errors_df1, url_errors_df2, url_errors_df3, url_errors_df4])
url_errors_df.shape

In [ ]:
print("Total errors:", url_errors_df.shape[0])
url_errors_df.tail()

In [ ]:
validity_by_url = pd.DataFrame(url_df_exploded.request_error.value_counts()).rename(columns={"count":"url_count"})
validity_by_file = pd.DataFrame(url_df_exploded.drop(columns=["urls", "valid_namespace_format", "valid_url_format"]).drop_duplicates().request_error.value_counts()).rename(columns={"count":"file_count"})
validity_stats = validity_by_url.join(validity_by_file, how="outer").reset_index()
validity_stats = validity_stats.rename(columns={"request_error":"url_error_type"})

In [ ]:
invalid_ref_urls = url_errors_df.loc[url_errors_df.valid_namespace_format == False].shape[0]
invalid_ref_files = url_errors_df.drop(columns=["urls"]).drop_duplicates()
invalid_ref_files = invalid_ref_files.loc[invalid_ref_files.valid_namespace_format == False].shape[0]
invalid_ref_df = pd.DataFrame({"url_error_type":["Invalid reference to namespace"], "url_count":[invalid_ref_urls], "file_count":[invalid_ref_files]})
validity_stats = pd.concat([validity_stats, invalid_ref_df], ignore_index=True)

invalid_ref_urls = url_errors_df.loc[url_errors_df.valid_url_format == False].shape[0]
invalid_ref_files = url_errors_df.drop(columns=["urls"]).drop_duplicates()
invalid_ref_files = invalid_ref_files.loc[invalid_ref_files.valid_url_format == False].shape[0]
invalid_ref_df = pd.DataFrame({"url_error_type":["Invalid URL format"], "url_count":[invalid_ref_urls], "file_count":[invalid_ref_files]})
validity_stats = pd.concat([validity_stats, invalid_ref_df], ignore_index=True)

validity_stats

##### All URLs
Next, extract all URLs included in the data, whether or not they're provided as a namespace.

In [ ]:
url_pattern = 'https?://[a-z0-9\-._~:/?#@!$&\'()*+,;=%]+'

In [ ]:
# Find all the URLs
files_with_urls, url_count_per_file, urls_per_file = [], [], []
for file_path in dublin_file_paths:
    with open(file_path, "r") as f:
        f_string = f.read().lower()
        
        # Look for URLs in the file
        has_urls = re.finditer(url_pattern, f_string)
        # Save the URLs in a list per file
        file_urls = []
        for match in has_urls:
            url = match[0]
            if " " in url:
                multiple = url.split(" ")
                file_urls = file_urls + multiple
                # print(file_urls)
            else:
                file_urls += [url]
        urls_per_file += [file_urls]
        url_count_per_file += [len(file_urls)]
        
        if len(file_urls) > 0:
            # Save the file path to the XML version of the file
            file_path.replace(".txt", ".xml")
            files_with_urls += [file_path]

        f.close()

print(sum(url_count_per_file), "URLs found in", len(files_with_urls), "files.")

In [ ]:
all_url_df = pd.DataFrame.from_dict({"file_path":dublin_file_paths, "url_count":url_count_per_file, "urls":urls_per_file}).sort_values(by="url_count", ascending=False)
all_url_df.head()

In [ ]:
all_url_df = all_url_df.loc[all_url_df["url_count"] > 0]  # Keep only files with URLs
all_url_df_exploded = all_url_df.explode("urls").drop(columns=["url_count"]).rename(columns={"urls":"url"})
all_url_df_exploded.head()

See if any new URLs that aren't namespaces (or intended to be namespaces) were found.

In [ ]:
# Create a new columns for both URL DataFrames with only the url, meaning every string should begin with http.
urls = list(url_df_exploded["urls"])
clean_urls = []
for url in urls:
    if "=" in url:
        clean_urls += [url.split('="')[-1].strip('"')]
    else:
        clean_urls += [url.strip('"')]
url_df_exploded.insert(2, "clean_url", clean_urls)
url_df_exploded.head()

In [ ]:
# Then compare the pairs of files and cleaned URLs to the newly extracted URL-file pairs by combining the two DataFrames, removing duplicates, and counting what's left
sub_url_df = url_df_exploded[["file_path", "clean_url"]]
urls = sub_url_df.join(all_url_df_exploded.set_index("file_path"), on="file_path", how="outer")
urls.head()

In [ ]:
print(urls.loc[urls.clean_url.isna()].shape)
print(urls.loc[urls.url.isna()].shape)

Look at the 12 newly found URLs (i.e., URLs that aren't namespaces).

In [ ]:
non_ns_urls = urls.loc[urls.clean_url.isna()]
non_ns_urls

Confirm that all URLs except the 12 above were already found as namespaces.

In [ ]:
clean_urls_list = list(urls.loc[~urls.clean_url.isna()].clean_url)
url_list = clean_urls_list = list(urls.loc[~urls.clean_url.isna()].url)
i, maxI = 0, len(clean_urls_list)
while i < maxI:
    assert clean_urls_list[0] == url_list[0]
    i += 1

Check whether each of the newly found URLs is a valid URL.

In [ ]:
request_errors = []
non_ns_url_list = list(non_ns_urls.url)
for url in non_ns_url_list:
        try:
            url_request = Request(url.strip(), headers=headers)
            html = urlopen(url_request, timeout=10).read()
            request_errors += ["No error"]  # Indicates a valid URL (though a manual check is needed to make sure it's a relevant URL)
        except Exception as e:
            request_errors += [str(e)]
print("Finished requests!")

In [ ]:
non_ns_urls = non_ns_urls.drop(columns=["clean_url"])
non_ns_urls.insert(len(non_ns_urls.columns), "request_error", request_errors)
non_ns_urls

In [ ]:
non_ns_urls_stats = pd.DataFrame(non_ns_urls.request_error.value_counts())
non_ns_urls_stats

Save the reports as CSV files.

In [ ]:
metadata_standard = "dublin_core"
data_serialization = "xml"

In [ ]:
report_type = "namespace_url_counts"
url_df.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=True
    )

In [ ]:
report_type = "namespace_url_validity_counts"
df_url_status.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=True
    )

In [ ]:
report_type = "namespace_url_errors"
url_df_exploded.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=True
    )

In [ ]:
report_type = "namespace_url_errors_stats"
validity_stats.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=True
    )

In [ ]:
report_type = "non-namespace_url_errors"
non_ns_urls.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=True
    )

In [ ]:
report_type = "non-namespace_url_errors_stats"
non_ns_urls_stats.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=True
    )

### JSON-LD

In [ ]:
extension = ".txt" #".json"
cidoc_file_paths = []
cidoc_files_t1 = [f for f in os.listdir(cidoc_t1_dir) if f.endswith(extension)]
cidoc_file_paths += [cidoc_t1_dir+f for f in cidoc_files_t1]
cidoc_files_p1 = [f for f in os.listdir(cidoc_p1_dir) if f.endswith(extension)]
cidoc_file_paths += [cidoc_p1_dir+f for f in cidoc_files_p1]
cidoc_files_p3 = [f for f in os.listdir(cidoc_p3_dir) if f.endswith(extension)]
cidoc_file_paths += [cidoc_p3_dir+f for f in cidoc_files_p3]
cidoc_file_paths.sort()
print("Total CIDOC-CRM JSON files:", len(cidoc_file_paths))

In [ ]:
cidoc_file_paths[0]

In [ ]:
extension = ".txt" #".json"
schema_file_paths = []
schema_files_t1 = os.listdir(schema_t1_dir)
schema_file_paths += [schema_t1_dir+f for f in schema_files_t1 if f.endswith(extension)]
schema_files_p1 = os.listdir(schema_p1_dir)
schema_file_paths += [schema_p1_dir+f for f in schema_files_p1 if f.endswith(extension)]
schema_files_p3 = os.listdir(schema_p3_dir)
schema_file_paths += [schema_p3_dir+f for f in schema_files_p3 if f.endswith(extension)]
schema_file_paths.sort()
print("Total Schema.org JSON files:", len(schema_file_paths))

In [ ]:
schema_file_paths[0]

In [ ]:
json_file_paths = cidoc_file_paths + schema_file_paths
total_json_files = len(json_file_paths)
print(len(json_file_paths))

#### Content of Fields
Check for empty metadata fields.

In [ ]:
# field_values = re.compile('((?<=:)\s*)"[^"]+"')
empty = re.compile('("[^"]+":\s?)(("(unknown|none|na|\?|not specified)")|"")')

First find the empty fields for the metadata records in CIDOC-CRM JSON-LD.

In [ ]:
files_with_empty, empty_fields_per_file, fields_per_file = utils.findEmptyFields(empty, cidoc_file_paths)

In [ ]:
df_cidoc_empty = pd.DataFrame.from_dict({"file_path":cidoc_file_paths, "model":["CIDOC-CRM"]*len(cidoc_file_paths), "empty_field_count":empty_fields_per_file, "fields":fields_per_file}).sort_values(by="empty_field_count", ascending=False)
df_cidoc_empty.head()

Next find the empty fields for the metadata records in Schema.org JSON-LD and add them to the `df_empty` DataFrame.

In [ ]:
files_with_empty, empty_fields_per_file, fields_per_file = utils.findEmptyFields(empty, schema_file_paths)

In [ ]:
df_sdo_empty = pd.DataFrame.from_dict({"file_path":schema_file_paths, "model":["Schema.org"]*len(schema_file_paths), "empty_field_count":empty_fields_per_file, "fields":fields_per_file}).sort_values(by="empty_field_count", ascending=False)
df_sdo_empty.head()

In [ ]:
df_empty = pd.concat([df_cidoc_empty, df_sdo_empty])
df_empty = df_empty.sort_values(by=["empty_field_count"], ascending=False)
df_empty.head()

"Explode" the DataFrame so that there is one row per field, rather than one row per file.  We'll exclude all the files that don't have any empty fields from this version of the data.

In [ ]:
df_empty_exploded = df_empty.loc[df_empty.empty_field_count > 0].drop(columns=["empty_field_count"]).explode("fields")
df_empty_exploded.head()

In [ ]:
df_empty_exploded[["field", "value"]] = df_empty_exploded["fields"].str.split(": ", expand=True)
df_empty_exploded.tail()

In [ ]:
df_model_fields = pd.DataFrame(df_empty_exploded.model.value_counts()).rename(columns={"count":"field_count"})
total_empty_fields = df_empty_exploded.shape[0]
df_model_files = pd.DataFrame(df_empty.model.value_counts()).rename(columns={"count":"file_count"})
total_files_with_empty = df_empty.shape[0]
df_model_totals = df_model_fields.join(df_model_files).reset_index()
df_model_totals = pd.concat([df_model_totals, pd.DataFrame.from_dict({"model": ["TOTAL"], "field_count": total_empty_fields, "file_count": total_files_with_empty})])
df_model_totals

In [ ]:
col = "field"
all_field_counts = pd.DataFrame(df_empty_exploded[col].value_counts())
sdo_field_counts = pd.DataFrame(df_empty_exploded.loc[df_empty_exploded.model == "Schema.org"][col].value_counts())
cidoc_field_counts = pd.DataFrame(df_empty_exploded.loc[df_empty_exploded.model == "CIDOC-CRM"][col].value_counts())
field_counts = all_field_counts.join(sdo_field_counts, rsuffix="_sdo_fields").join(cidoc_field_counts, rsuffix="_cidoc_fields")
field_counts = field_counts.rename(columns={"count":"field_count"})
# field_counts
subdf = df_empty_exploded[["file_path", "model", col]].drop_duplicates()
all_file_counts = pd.DataFrame(subdf[col].value_counts())
sdo_file_counts = pd.DataFrame(subdf.loc[subdf.model == "Schema.org"][col].value_counts())
cidoc_file_counts = pd.DataFrame(subdf.loc[subdf.model == "CIDOC-CRM"][col].value_counts())
file_counts = all_file_counts.join(sdo_file_counts, rsuffix="_sdo_files").join(cidoc_file_counts, rsuffix="_cidoc_files")
file_counts = file_counts.rename(columns={"count":"file_count"})
# file_counts
field_counts = field_counts.join(file_counts)
field_counts = field_counts.fillna(0)
field_counts


In [ ]:
col = "value"
all_value_counts = pd.DataFrame(df_empty_exploded[col].value_counts())
sdo_value_counts = pd.DataFrame(df_empty_exploded.loc[df_empty_exploded.model == "Schema.org"][col].value_counts())
cidoc_value_counts = pd.DataFrame(df_empty_exploded.loc[df_empty_exploded.model == "CIDOC-CRM"][col].value_counts())
value_counts = all_value_counts.join(sdo_value_counts, rsuffix="_sdo_fields").join(cidoc_value_counts, rsuffix="_cidoc_fields")
value_counts = value_counts.rename(columns={"count":"field_count"})
# value_counts
subdf = df_empty_exploded[["file_path", "model", col]].drop_duplicates()
all_file_counts = pd.DataFrame(subdf[col].value_counts())
sdo_file_counts = pd.DataFrame(subdf.loc[subdf.model == "Schema.org"][col].value_counts())
cidoc_file_counts = pd.DataFrame(subdf.loc[subdf.model == "CIDOC-CRM"][col].value_counts())
file_counts = all_file_counts.join(sdo_file_counts, rsuffix="_sdo_files").join(cidoc_file_counts, rsuffix="_cidoc_files")
file_counts = file_counts.rename(columns={"count":"file_count"})
# file_counts
value_counts = value_counts.join(file_counts)
value_counts = value_counts.fillna(0)

# Reformat the empty values column to make sure empty strings ("") or newlines (\n) are visible in the CSV file of this report
value_counts = utils.emptyErrorReportValues(value_counts, "value")

value_counts


Save the reports as CSV files.

In [ ]:
metadata_standard = "sdo-cidoc"
data_serialization = "json-ld"

In [ ]:
report_type = "empty_fields_by_file"
df_empty.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=True
    )

In [ ]:
report_type = "empty_fields_by_field"
df_empty_exploded.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=True
    )

In [ ]:
report_type = "empty_by_model"
df_model_totals.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=True
    )

In [ ]:
report_type = "empty_field_counts"
field_counts.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=True
    )

In [ ]:
report_type = "empty_value_counts"
value_counts.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=True
    )

#### URLs
Check that URLs are well-formed and that they exist.

In [ ]:
url_pattern = 'https?://[a-z0-9\-._~:/?#@!$&\'()*+,;=%]+'

In [ ]:
# Find all the URLs
files_with_urls, url_count_per_file, urls_per_file = [], [], []
for file_path in schema_file_paths:
    with open(file_path, "r") as f:
        f_string = f.read().lower()
        
        # Look for URLs in the file
        has_urls = re.finditer(url_pattern, f_string)
        # Save the URLs in a list per file
        file_urls = []
        for match in has_urls:
            url = match[0]
            if " " in url:
                multiple = url.split(" ")
                file_urls = file_urls + multiple
                # print(file_urls)
            else:
                file_urls += [url]
        urls_per_file += [file_urls]
        url_count_per_file += [len(file_urls)]
        
        if len(file_urls) > 0:
            # Save the file path to the XML version of the file
            file_path.replace(".txt", ".xml")
            files_with_urls += [file_path]

        f.close()

print(sum(url_count_per_file), "URLs found in", len(files_with_urls), "files.")

In [ ]:
sdo_url_df = pd.DataFrame.from_dict({"file_path":schema_file_paths, "model": ["Schema.org"]*len(schema_file_paths), "url_count":url_count_per_file, "urls":urls_per_file}).sort_values(by="url_count", ascending=False)

In [ ]:
# Find all the URLs
files_with_urls, url_count_per_file, urls_per_file = [], [], []
for file_path in cidoc_file_paths:
    with open(file_path, "r") as f:
        f_string = f.read().lower()
        
        # Look for URLs in the file
        has_urls = re.finditer(url_pattern, f_string)
        # Save the URLs in a list per file
        file_urls = []
        for match in has_urls:
            url = match[0]
            if " " in url:
                multiple = url.split(" ")
                file_urls = file_urls + multiple
                # print(file_urls)
            else:
                file_urls += [url]
        urls_per_file += [file_urls]
        url_count_per_file += [len(file_urls)]
        
        if len(file_urls) > 0:
            # Save the file path to the XML version of the file
            file_path.replace(".txt", ".xml")
            files_with_urls += [file_path]

        f.close()

print(sum(url_count_per_file), "URLs found in", len(files_with_urls), "files.")

In [ ]:
cidoc_url_df = pd.DataFrame.from_dict({"file_path":cidoc_file_paths, "model": ["CIDOC-CRM"]*len(cidoc_file_paths), "url_count":url_count_per_file, "urls":urls_per_file}).sort_values(by="url_count", ascending=False)

In [ ]:
url_df = pd.concat([sdo_url_df, cidoc_url_df])
url_df = url_df.sort_values(by="url_count", ascending=False)
url_df.head()

In [ ]:
url_df = url_df.loc[url_df["url_count"] > 0]  # Keep only files with URLs
url_df_exploded = url_df.explode("urls").drop(columns=["url_count"])
url_df_exploded.head()

Check whether each of the newly found URLs is a valid URL.

In [ ]:
request_errors = []
json_url_list = list(url_df_exploded.urls)
for url in json_url_list:
        try:
            url_request = Request(url.strip(), headers=headers)
            html = urlopen(url_request, timeout=10).read()
            request_errors += ["No error"]  # Indicates a valid URL (though a manual check is needed to make sure it's a relevant URL)
        except Exception as e:
            request_errors += [str(e)]
print("Finished requests!")

In [ ]:
url_df_exploded.insert(len(url_df_exploded.columns), "request_error", request_errors)
url_df_exploded.head()

In [ ]:
all_urls_df = pd.DataFrame.from_dict({"model":["TOTAL"], "url_count":[url_df_exploded.shape[0]], "file_count":[url_df.shape[0]]}).set_index("model")
urls_model_df = pd.DataFrame(url_df_exploded.model.value_counts()).rename(columns={"count":"url_count"})
files_model_df = pd.DataFrame(url_df.model.value_counts()).rename(columns={"count":"file_count"})
model_df = urls_model_df.join(files_model_df)
model_df = pd.concat([model_df, all_urls_df])
model_df

In [ ]:
error_stats = pd.DataFrame(url_df_exploded.request_error.value_counts())
cidoc_error_stats = pd.DataFrame(url_df_exploded.loc[url_df_exploded.model == "CIDOC-CRM"].request_error.value_counts()).rename(columns={"count":"cidoc-crm_count"})
schema_error_stats = pd.DataFrame(url_df_exploded.loc[url_df_exploded.model == "Schema.org"].request_error.value_counts()).rename(columns={"count":"schema-org_count"})
error_stats = error_stats.join(cidoc_error_stats).join(schema_error_stats).fillna(0)

all_errors = url_df_exploded.loc[url_df_exploded.request_error != "No error"]
cidoc_errors = all_errors.loc[all_errors.model == "CIDOC-CRM"].shape[0]
schema_errors = all_errors.loc[all_errors.model == "Schema.org"].shape[0]
total_errors = pd.DataFrame.from_dict({"request_error":["ALL REQUEST ERRORS"], "count":[all_errors.shape[0]], "cidoc-crm_count":[cidoc_errors], "schema-org_count":[schema_errors]}).set_index("request_error")

error_stats = pd.concat([error_stats, total_errors])
error_stats

Save the reports as CSV files.

In [ ]:
metadata_standard = "sdo-cidoc"
data_serialization = "json-ld"

In [ ]:
report_type = "url_counts_per_file"
url_df.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=True
    )

In [ ]:
report_type = "url_counts_per_model"
model_df.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=True
    )

In [ ]:
report_type = "urls" # includes column for request errors
url_df_exploded.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=True
    )

In [ ]:
report_type = "url_errors"
all_errors.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=True
    )

In [ ]:
report_type = "url_errors_stats"
error_stats.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=True
    )